# All imports #

In [107]:
from __future__ import print_function

import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import tensorflow.keras
import pandas as pd
import numpy as np
import sklearn
import keras
import csv

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input

from keras_uncertainty.models import MCDropoutClassifier, MCDropoutRegressor
from keras_uncertainty.utils import numpy_regression_nll
from keras_uncertainty.models import DeepEnsembleRegressor, deep_ensemble_regression_nll_loss



In [108]:
def load_joint_space_csv_chunks(file_path):
    data_frame = pd.read_csv(file_path, skiprows=1, header=None)
    del data_frame[18]
    return data_frame

def load_task_space_csv_chunks(file_path):
    return pd.read_csv(file_path, skiprows=1, header=None)


In [109]:
##please select the appropriate folder, willl use os.path.join() for completed script
TRAIN_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/'
TEST_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/test_4steps'

X_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_sysstate_q.csv')
x_train = load_task_space_csv_chunks(X_TRAIN_FILE)
q_train = load_joint_space_csv_chunks(Q_TRAIN_FILE)

X_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_sysstate_q.csv')
x_test = load_task_space_csv_chunks(X_TEST_FILE)
q_test = load_joint_space_csv_chunks(Q_TEST_FILE)

In [110]:
x_train = load_task_space_csv_chunks(X_TRAIN_FILE)
q_train = load_joint_space_csv_chunks(Q_TRAIN_FILE)

x_test = load_task_space_csv_chunks(X_TEST_FILE)
q_test = load_joint_space_csv_chunks(Q_TEST_FILE)

x_scaler = MinMaxScaler()
q_scaler = MinMaxScaler()

x_train = x_scaler.fit_transform(x_train)
q_train = q_scaler.fit_transform(q_train)

x_test = x_scaler.transform(x_test)
q_test = q_scaler.transform(q_test)

In [111]:

def model_builder():
    input_shape = 9
    output_shape = 18
    depth = 6
    width = 64
    reduction_factor = 0.9
    for i in range(depth):
        num_neurons = max(int(width * (reduction_factor ** i)), 4)
        if i == 0:
            inp = Input(shape=(input_shape,))
            x = Dense(num_neurons, activation='relu')(inp)
        else:
            x = Dense(num_neurons, activation='relu')(x)
   ##mean and variance are the extra layers added to the OG model to make it O/P Variance. 
    mean = Dense(output_shape, activation="sigmoid")(x)
    var = Dense(output_shape, activation="softplus")(x)
    
    train_model = Model(inp, mean)
    pred_model = Model(inp, [mean, var])

    train_model.compile(loss=deep_ensemble_regression_nll_loss(var), optimizer="adam", metrics=["mae"])
    return train_model, pred_model

In [112]:


model = model_builder

print(type(model))

<class 'function'>


In [113]:

##using cpu core-0

with tf.device('/cpu:0'):
    ensemble_model = DeepEnsembleRegressor(model_builder, 15)
    ensemble_model.fit(x_train, q_train, epochs = 100, batch_size = 128, verbose = 1, validation_data=(x_test,q_test))

Train on 15625 samples, validate on 4096 samples
Epoch 1/100
15625/15625 [==============================] - 7s 419us/sample - loss: -0.3317 - mean_absolute_error: 0.2782 - val_loss: -0.4787 - val_mean_absolute_error: 0.2694
Epoch 2/100
15625/15625 [==============================] - 1s 71us/sample - loss: -0.5468 - mean_absolute_error: 0.2544 - val_loss: -0.5976 - val_mean_absolute_error: 0.2447
Epoch 3/100
15625/15625 [==============================] - 1s 70us/sample - loss: -0.7099 - mean_absolute_error: 0.2225 - val_loss: -0.7796 - val_mean_absolute_error: 0.2100
Epoch 4/100
15625/15625 [==============================] - 1s 69us/sample - loss: -0.8535 - mean_absolute_error: 0.1992 - val_loss: -0.9344 - val_mean_absolute_error: 0.1851
Epoch 5/100
15625/15625 [==============================] - 1s 70us/sample - loss: -0.9807 - mean_absolute_error: 0.1775 - val_loss: -1.1140 - val_mean_absolute_error: 0.1643
Epoch 6/100
15625/15625 [==============================] - 1s 70us/sample - loss

Epoch 94/100
15625/15625 [==============================] - 1s 72us/sample - loss: -1.7515 - mean_absolute_error: 0.1068 - val_loss: -1.7159 - val_mean_absolute_error: 0.1073
Epoch 95/100
15625/15625 [==============================] - 1s 71us/sample - loss: -1.7523 - mean_absolute_error: 0.1068 - val_loss: -1.7720 - val_mean_absolute_error: 0.1065
Epoch 96/100
15625/15625 [==============================] - 1s 73us/sample - loss: -1.7662 - mean_absolute_error: 0.1061 - val_loss: -1.7755 - val_mean_absolute_error: 0.1057
Epoch 97/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.7711 - mean_absolute_error: 0.1058 - val_loss: -1.6118 - val_mean_absolute_error: 0.1077
Epoch 98/100
15625/15625 [==============================] - 1s 73us/sample - loss: -1.7662 - mean_absolute_error: 0.1063 - val_loss: -1.6505 - val_mean_absolute_error: 0.1081
Epoch 99/100
15625/15625 [==============================] - 1s 77us/sample - loss: -1.7745 - mean_absolute_error: 0.1059 - va

Epoch 87/100
15625/15625 [==============================] - 1s 73us/sample - loss: -2.1632 - mean_absolute_error: 0.0442 - val_loss: -0.5269 - val_mean_absolute_error: 0.0577
Epoch 88/100
15625/15625 [==============================] - 1s 70us/sample - loss: -2.1040 - mean_absolute_error: 0.0452 - val_loss: -1.6820 - val_mean_absolute_error: 0.0489
Epoch 89/100
15625/15625 [==============================] - 1s 69us/sample - loss: -2.1451 - mean_absolute_error: 0.0444 - val_loss: -1.7930 - val_mean_absolute_error: 0.0481
Epoch 90/100
15625/15625 [==============================] - 1s 71us/sample - loss: -2.1785 - mean_absolute_error: 0.0438 - val_loss: -1.2695 - val_mean_absolute_error: 0.0512
Epoch 91/100
15625/15625 [==============================] - 1s 71us/sample - loss: -2.1556 - mean_absolute_error: 0.0442 - val_loss: -1.2639 - val_mean_absolute_error: 0.0516
Epoch 92/100
15625/15625 [==============================] - 1s 70us/sample - loss: -2.1459 - mean_absolute_error: 0.0442 - va

Epoch 80/100
15625/15625 [==============================] - 1s 74us/sample - loss: -2.0416 - mean_absolute_error: 0.0628 - val_loss: -1.5167 - val_mean_absolute_error: 0.0636
Epoch 81/100
15625/15625 [==============================] - 1s 75us/sample - loss: -2.0672 - mean_absolute_error: 0.0622 - val_loss: -1.2884 - val_mean_absolute_error: 0.0693
Epoch 82/100
15625/15625 [==============================] - 1s 73us/sample - loss: -2.0648 - mean_absolute_error: 0.0620 - val_loss: -1.2600 - val_mean_absolute_error: 0.0655
Epoch 83/100
15625/15625 [==============================] - 1s 72us/sample - loss: -2.0665 - mean_absolute_error: 0.0618 - val_loss: -1.3798 - val_mean_absolute_error: 0.0635
Epoch 84/100
15625/15625 [==============================] - 1s 72us/sample - loss: -2.0817 - mean_absolute_error: 0.0613 - val_loss: -1.2963 - val_mean_absolute_error: 0.0644
Epoch 85/100
15625/15625 [==============================] - 1s 73us/sample - loss: -2.0780 - mean_absolute_error: 0.0614 - va

15625/15625 [==============================] - 1s 74us/sample - loss: -1.9715 - mean_absolute_error: 0.1218 - val_loss: 4.0838 - val_mean_absolute_error: 0.1258
Epoch 74/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.9749 - mean_absolute_error: 0.1212 - val_loss: 5.2335 - val_mean_absolute_error: 0.1290
Epoch 75/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.4842 - mean_absolute_error: 0.1369 - val_loss: 1.5698 - val_mean_absolute_error: 0.1274
Epoch 76/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.9439 - mean_absolute_error: 0.1220 - val_loss: 4.6873 - val_mean_absolute_error: 0.1262
Epoch 77/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.9592 - mean_absolute_error: 0.1213 - val_loss: 4.7771 - val_mean_absolute_error: 0.1337
Epoch 78/100
15625/15625 [==============================] - 1s 76us/sample - loss: -1.9723 - mean_absolute_error: 0.1211 - val_loss: 4.6380 - v

...

Epoch 67/100
15625/15625 [==============================] - 1s 76us/sample - loss: -1.8698 - mean_absolute_error: 0.0626 - val_loss: -1.8229 - val_mean_absolute_error: 0.0640
Epoch 68/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.8714 - mean_absolute_error: 0.0625 - val_loss: -1.7799 - val_mean_absolute_error: 0.0648
Epoch 69/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.8709 - mean_absolute_error: 0.0622 - val_loss: -1.7709 - val_mean_absolute_error: 0.0642
Epoch 70/100
15625/15625 [==============================] - 1s 76us/sample - loss: -1.8853 - mean_absolute_error: 0.0621 - val_loss: -1.7631 - val_mean_absolute_error: 0.0643
Epoch 71/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.8412 - mean_absolute_error: 0.0624 - val_loss: -1.7203 - val_mean_absolute_error: 0.0648
Epoch 72/100
15625/15625 [==============================] - 1s 76us/sample - loss: -1.8829 - mean_absolute_error: 0.0618 - va

Epoch 60/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.8327 - mean_absolute_error: 0.0792 - val_loss: -1.8273 - val_mean_absolute_error: 0.0777
Epoch 61/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.8774 - mean_absolute_error: 0.0783 - val_loss: -1.0615 - val_mean_absolute_error: 0.0803
Epoch 62/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.8501 - mean_absolute_error: 0.0785 - val_loss: -1.7108 - val_mean_absolute_error: 0.0814
Epoch 63/100
15625/15625 [==============================] - 1s 76us/sample - loss: -1.8703 - mean_absolute_error: 0.0782 - val_loss: -1.8932 - val_mean_absolute_error: 0.0754
Epoch 64/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.8796 - mean_absolute_error: 0.0780 - val_loss: -1.8090 - val_mean_absolute_error: 0.0784
Epoch 65/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.8768 - mean_absolute_error: 0.0781 - va

Epoch 53/100
15625/15625 [==============================] - 1s 75us/sample - loss: -1.9199 - mean_absolute_error: 0.0883 - val_loss: -1.6322 - val_mean_absolute_error: 0.0907
Epoch 54/100
15625/15625 [==============================] - 1s 73us/sample - loss: -1.9245 - mean_absolute_error: 0.0884 - val_loss: -1.6607 - val_mean_absolute_error: 0.0941
Epoch 55/100
15625/15625 [==============================] - 1s 73us/sample - loss: -1.9328 - mean_absolute_error: 0.0877 - val_loss: -1.7116 - val_mean_absolute_error: 0.0908
Epoch 56/100
15625/15625 [==============================] - 1s 72us/sample - loss: -1.9425 - mean_absolute_error: 0.0874 - val_loss: -1.6487 - val_mean_absolute_error: 0.0908
Epoch 57/100
15625/15625 [==============================] - 1s 73us/sample - loss: -1.9348 - mean_absolute_error: 0.0876 - val_loss: -1.6048 - val_mean_absolute_error: 0.0924
Epoch 58/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.9399 - mean_absolute_error: 0.0875 - va

Epoch 100/100
15625/15625 [==============================] - 1s 75us/sample - loss: -2.0631 - mean_absolute_error: 0.0821 - val_loss: -1.2967 - val_mean_absolute_error: 0.0922
Train on 15625 samples, validate on 4096 samples
Epoch 1/100
15625/15625 [==============================] - 7s 460us/sample - loss: -0.4389 - mean_absolute_error: 0.2802 - val_loss: -0.6028 - val_mean_absolute_error: 0.2634
Epoch 2/100
15625/15625 [==============================] - 1s 73us/sample - loss: -0.6958 - mean_absolute_error: 0.2375 - val_loss: -0.7929 - val_mean_absolute_error: 0.2159
Epoch 3/100
15625/15625 [==============================] - 1s 73us/sample - loss: -0.8575 - mean_absolute_error: 0.2026 - val_loss: -0.9336 - val_mean_absolute_error: 0.1861
Epoch 4/100
15625/15625 [==============================] - 1s 74us/sample - loss: -1.0038 - mean_absolute_error: 0.1810 - val_loss: -1.0982 - val_mean_absolute_error: 0.1719
Epoch 5/100
15625/15625 [==============================] - 1s 73us/sample - lo

Epoch 93/100
15625/15625 [==============================] - 1s 82us/sample - loss: -2.2652 - mean_absolute_error: 0.0543 - val_loss: 1.6508 - val_mean_absolute_error: 0.0542
Epoch 94/100
15625/15625 [==============================] - 1s 86us/sample - loss: -2.2629 - mean_absolute_error: 0.0540 - val_loss: 0.6538 - val_mean_absolute_error: 0.0556
Epoch 95/100
15625/15625 [==============================] - 1s 76us/sample - loss: -2.2660 - mean_absolute_error: 0.0541 - val_loss: 0.9171 - val_mean_absolute_error: 0.0541
Epoch 96/100
15625/15625 [==============================] - 1s 82us/sample - loss: -2.2674 - mean_absolute_error: 0.0539 - val_loss: 1.0055 - val_mean_absolute_error: 0.0573
Epoch 97/100
15625/15625 [==============================] - 1s 78us/sample - loss: -2.2516 - mean_absolute_error: 0.0544 - val_loss: 1.2875 - val_mean_absolute_error: 0.0545
Epoch 98/100
15625/15625 [==============================] - 1s 82us/sample - loss: -2.2762 - mean_absolute_error: 0.0538 - val_los

In [114]:
mean_test, sd_test = ensemble_model.predict(x_test)
mean_test = q_scaler.inverse_transform(mean_test)
q_test_unorm = q_scaler.inverse_transform(q_test)
print("NLL: {:.5f}".format(numpy_regression_nll(q_test_unorm, mean_test, sd_test**2)))

NLL: -0.46288


In [48]:
print(tf.__version__)

1.13.1
